In [158]:
import cartoframes
import requests
import pandas as pd
import geopandas as gpd
import json
import shapely

from carto.auth import APIKeyAuthClient
from carto.sql import SQLClient
from carto.datasets import DatasetManager

In [32]:
cc = cartoframes.CartoContext()

In [33]:
auth_client = APIKeyAuthClient(api_key=cc.creds.key(), base_url=cc.creds.base_url())

In [34]:
# mapzen api_key
api_key = 'mapzen-CQ12NPK'

In [35]:
brooklyn = '102082361'
crown_heights = '85867069'

In [171]:
getDescendantsURL = 'https://places.mapzen.com/v1/?method=mapzen.places.getDescendants&api_key={api_key}&id={id}&placetype={placetype}'
getInfoURL = 'https://places.mapzen.com/v1/?method=mapzen.places.getInfo&api_key={api_key}&&id={ids}&format=json&extras=mz:uri'
getInfoMultiURL = 'https://places.mapzen.com/v1/?method=mapzen.places.getInfoMulti&api_key={api_key}&&ids={ids}&format=csv&extras=mz:uri'
placesSearchURL = 'https://places.mapzen.com/v1/?method=mapzen.places.search&api_key={api_key}&names={names}&placetype={placetype}&format=csv&per_page=500'
getPlaceTypesList = 'https://places.mapzen.com/v1/?method=mapzen.places.placetypes.getList&api_key={api_key}&role=common'

In [282]:
requests.get(getInfoURL.format(api_key=api_key,ids='102082361'))

<Response [200]>

In [283]:
getInfoURL.format(api_key=api_key,ids='102082361')

'https://places.mapzen.com/v1/?method=mapzen.places.getInfo&api_key=mapzen-CQ12NPK&&id=102082361&format=json&extras=mz:uri'

In [172]:
placesSearchURL.format(api_key=api_key, names='',placetype='country')

'https://places.mapzen.com/v1/?method=mapzen.places.search&api_key=mapzen-CQ12NPK&names=&placetype=country&format=csv&per_page=500'

In [173]:
countries = pd.read_csv(placesSearchURL.format(api_key=api_key, names='',placetype='country'))

In [207]:
alphabetical = countries['wof_name'].sort_values()

In [256]:
brazil = countries.iloc[5]

In [253]:
countries = pd.read_csv(placesSearchURL.format(api_key=api_key, names='',placetype='country'))

wof_country                    BR
wof_id                   85633009
wof_name                   Brazil
wof_parent_id           102191577
wof_placetype             country
wof_repo         whosonfirst-data
Name: 5, dtype: object

In [258]:
brazil_region_url = placesSearchURL.format(api_key=api_key, names='',placetype='region')+'&country_id={}'.format(brazil.wof_id)

In [264]:
brazil_regions = pd.read_csv(brazil_region_url)

In [267]:
brazil_regions.shape

(27, 6)

In [273]:
for brazil_region_id in brazil_regions.wof_id:
    response = requests.get(getInfoURL.format(api_key=api_key,ids=brazil_region_id))
    data = json.loads(response.text)
    geojson_url = data['place']['mz:uri']
    polygon = json.loads(requests.get(geojson_url).text)
    plt.shapely.geometry.shape(polygon['geometry'])

In [114]:
response = requests.get(placesSearchURL.format(api_key=api_key,names=['New York'],placetypes=''))

In [115]:
data = json.loads(response.text)

In [117]:
#grab the first place
feeling_lucky_id = data['places'][0]['wof:id']

In [151]:
response = requests.get(getInfoURL.format(api_key=api_key,ids=feeling_lucky_id))
data = json.loads(response.text)
geojson_url = data['place']['mz:uri']
polygon = json.loads(requests.get(geojson_url).text)

In [25]:
# Send the dataset to CARTO
dataset_manager = DatasetManager(auth_client)
dataset = dataset_manager.create(geojson_url)

/Users/mho/anaconda3/lib/python3.5/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)


In [44]:
# some WOF properties to keep
geom = polygon['geometry']
name = polygon['properties']['wof:label']
wof_id = polygon['properties']['wof:id']

In [119]:
response = requests.get(getDescendantsURL.format(api_key=api_key,id=feeling_lucky_id,placetype=''))
data = json.loads(response.text)
feeling_lucky_id

1108746619